In [293]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [294]:
np.random.seed(7)

In [295]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2021-12-08 23:40:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0s      

2021-12-08 23:40:47 (209 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [296]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [297]:
### Convert to date time object

In [298]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


In [299]:
# Data visualization and pre-processing

In [300]:
df['loan_status'].value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

In [ ]:
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
# Pre-processing:  Feature selection/extraction

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

In [ ]:
## Convert Categorical features to numerical values

In [ ]:
####Let's convert male to 0 and female to 1:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

In [ ]:
## One Hot Encoding
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

In [ ]:
#### Features before One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

In [ ]:
#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

In [ ]:
### Feature Selection

In [ ]:
#####Let's define feature sets, X:

In [ ]:
X = Feature
X[0:5]

In [ ]:
y = df['loan_status'].replace(to_replace=['PAIDOFF', 'COLLECTION'], value=[0,1]).values
y[0:5]

In [ ]:
# Normalize Data

In [ ]:
#Data Standardization give data zero mean and unit variance (technically should be done after train test split)

In [ ]:
X=preprocessing.StandardScaler().fit_transform(X.astype(float))
X[0:5]

In [ ]:
# Classification

from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)
score_matrix={}

In [ ]:
# Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
#You should use the following algorithm:

#   K Nearest Neighbor(KNN)
#   Decision Tree
#   Support Vector Machine
#   Logistic Regression


In [ ]:
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsClassifier
K = 15
mean_acc = np.zeros((K-1))
std_acc = np.zeros((K-1))

for n in range(1,K):
    
    #Train Model and Predict  
    knn_clf = KNeighborsClassifier(n_neighbors = n, metric='euclidean').fit(X_train,y_train)
    y_hat=knn_clf.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, y_hat)

    
    std_acc[n-1]=np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0])

mean_acc
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

In [ ]:
y_hat=knn_clf.predict(X_test)
score_matrix['knn-f1-score']=f1_score(y_test, y_hat, average='micro')
score_matrix['knn-jaccord']=jaccard_score(y_test, y_hat, pos_label = 0)

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(criterion="entropy", max_depth = 6).fit(X_train,y_train)



In [ ]:
y_hat = tree_clf.predict(X_test)
score_matrix['Tree-f1-score']=f1_score(y_test, y_hat, average='micro')
score_matrix['Tree-jaccord']=jaccard_score(y_test, y_hat, pos_label=0)

In [ ]:
#SVM
from sklearn import svm
svm_clf = svm.SVC(C=1000.0, kernel='rbf', gamma='scale',).fit(X_train, y_train)
y_hat=svm_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

cnf_matrix = confusion_matrix(y_test, y_hat, labels=[0,1])
np.set_printoptions(precision=2)

print (classification_report(y_test, y_hat))
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['PAIDOFF','COLLECTION'],normalize= False,  title='Confusion matrix')

In [ ]:
score_matrix['SVM-f1-score']=f1_score(y_test, y_hat, pos_label=0, average='micro')
score_matrix['SVM-jaccord']=jaccard_score(y_test, y_hat, pos_label=0)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
LR_clf = LogisticRegression(C=0.001, solver='liblinear', multi_class='auto').fit(X_train,y_train)
y_hat = LR_clf.predict(X_test)
y_hat
y_proba = LR_clf.predict_proba(X_test)

In [ ]:
score_matrix['LogReg-f1-score']=f1_score(y_test, y_hat, average='micro')
score_matrix['LogReg-jaccord']=jaccard_score(y_test, y_hat, pos_label=0)
score_matrix['LogReg-log-loss']=log_loss(y_test, y_proba)

In [ ]:
score_matrix

In [ ]:
# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_score, f1_score, log_loss

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

In [ ]:
### Load Test set for evaluation
test_df = pd.read_csv('loan_test.csv')
test_df.shape

In [ ]:
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_Feature = test_df[['Principal','terms','age','Gender','weekend']]
test_Feature = pd.concat([test_Feature,pd.get_dummies(test_df['education'])], axis=1)
test_Feature.drop(['Master or Above'], axis = 1,inplace=True)
X_test=preprocessing.StandardScaler().fit(test_Feature).transform(test_Feature)
X_test[0:5]
y_test = test_df['loan_status'].values
y_test[0:5]

In [ ]:
Report = {'Algorithm': ["KNN","Decision_Tree","SVM","LogisticRegression"], 
          'Jaccard': [score_matrix['Tree-jaccord'],score_matrix['Tree-jaccord'],score_matrix['SVM-jaccord'],score_matrix['LogReg-jaccord']],
          'F1_score': [score_matrix['knn-f1-score'], score_matrix['Tree-f1-score'],score_matrix['SVM-f1-score'],score_matrix['LogReg-f1-score']],
          'LogLoss': ["NA", "NA","NA",score_matrix['LogReg-log-loss']]}
                      
df_report = pd.DataFrame(data=Report)
df_report